In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import regex as re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    step7 = step6.replace("rt", "")
    return step7.strip()

In [ ]:
def sentiment_analyze(sent_token):

    sentence = untokenize(sent_token)

    sentiment_dict = analyzer.polarity_scores(sentence)

    if sentiment_dict['compound'] > 0.0 :
        return 1
 
    elif sentiment_dict['compound'] < -0.0 :
        return -1

    else :
      return 0

In [ ]:
import json
from google.colab import files

tweets = []
labels = []

for line in open('/content/drive/MyDrive/ML Project/Dataset/stocknet-dataset-master/tweet/preprocessed/AAPL/2014-01-01', 'r'):
    tweet_dict = json.loads(line)
    tweet_text = tweet_dict['text'] 
    tweets.append(tweet_text)
    labels.append(sentiment_analyze(tweet_text))

print(tweets)
label_df = pd.DataFrame(list(labels), columns =['SENTIMENT'])
label_df.to_csv('try.csv') 
files.download('try.csv')


In [ ]:
print(labels)

In [ ]:
comp = 'MSFT'

In [ ]:
import os
# Parent Directory path
parent_dir = "/content/drive/MyDrive/ML Project/Dataset/Sentiments/"
  
# Path
path = os.path.join(parent_dir, comp)

os.mkdir(path)

In [ ]:
def sentiment_file(file_path, date):

  labels = []

  for line in open(file_path, 'r'):
    tweet_dict = json.loads(line)
    tweet_text = tweet_dict['text'] 
    labels.append(sentiment_analyze(tweet_text))

  pos = labels.count(1)
  neut = labels.count(0)
  neg = labels.count(-1)

  return pos, neut, neg

  # label_df = pd.DataFrame(list(labels), columns =['SENTIMENT'])
  # label_df.to_csv('/content/drive/MyDrive/ML Project/Dataset/Sentiments/' + comp +'/'+date+'.csv')


In [ ]:
import os

# Define the location of the directory
path = r"/content/drive/MyDrive/ML Project/Dataset/stocknet-dataset-master/tweet/preprocessed/"+comp

# Change the directory
os.chdir(path)

date = []
positive = []
neutral = []
negative = []

# Iterate over all the files in the directory
for file in sorted(os.listdir()):
      print(str(file))
      file_path =f"{path}/{file}"
      pos, neut, neg = sentiment_file(file_path, str(file))
      date.append(str(file))
      positive.append(pos)
      neutral.append(neut)
      negative.append(neg)

a = np.sum(np.array(positive))
b = np.sum(np.array(neutral))
c = np.sum(np.array(negative))

# stock_df = pd.DataFrame(
#     {'DATE': date,
#      'POSITIVE': positive,
#      'NEUTRAL': neutral,
#      'NEGATIVE' : negative
#     })

# label_df = pd.DataFrame(list(zip(labels), columns =['SENTIMENT'])
#stock_df.to_csv('/content/drive/MyDrive/ML Project/Dataset/Sentiments/' + comp +'/'+'0000-00-00'+'.csv')


In [ ]:
print(a,b,c)

In [ ]:
import plotly.graph_objects as go

labels = ['Positive','Neutral','Negative']
values = [a,b,c]

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.show()

In [ ]:
# labels = [1,0,1]
# date = "2015-10-22"
# label_df = pd.DataFrame(list(labels), columns =['SENTIMENT'])
# label_df.to_csv('/content/drive/MyDrive/ML Project/Dataset/Sentiments/AAPL/'+date+'.csv')